In [ ]:
from random import *
from math   import *

In [ ]:
!mkdir outputs

In [ ]:
toplevel_file = lambda algo, coef_w, coefs :f"""
module Top where

import ConfigTH
import Graph.SingleConstantMult
import Graph.Aop
import Graph.Util
import Graph.MCM
import Graph.Pipelined
import MultBlock
import Clash.Prelude

graphDesc = $(mcmPipelinedTH {algo} {coefs})

multB = uncurry mkVecHw graphDesc

fir x = register 0 $ foldl f 0 (multB $ register 0 x)
  where f ac x = register 0 $ ac + x

createDomain vSystem{{vName="SystemNR", vResetPolarity=ActiveLow}}

topEntity
  :: Clock  SystemNR
  -> Reset  SystemNR
  -> Signal SystemNR (Unsigned 16)
  -> Signal SystemNR (Unsigned (16+{coef_w}+CLog 2 {len(coefs)}))
topEntity c r x = bundle $ exposeClockResetEnable (fir $ fmap resize x) c r (toEnable $ pure True)


{{-# NOINLINE topEntity #-}}
{{-# ANN topEntity
  (Synthesize
    {{ t_name   = "fir_rsg"
    , t_inputs = [ PortName "clk"
                 , PortName "rst"
                 , PortName "x" ]
    , t_output = PortName "y"
    }}) #-}}
"""

In [ ]:
def gen_coefs(b,n):
    return [ randint(1, 2**b-1) for _ in range(n)]

In [ ]:
def run_test(algo, n, coefs, coef_w, run):
    name = f'mcm_{algo}_n{n}_width{coef_w}_run{run}'
    print("Starting run for " + name)

    !make clean

    print("Building clash")
    !make build_dir
    with open('./build/clash/TopLevel.hs', 'w') as f:
        f.write(toplevel_file(algo,coef_w,coefs))
    !make verilog > ./build/xil/clash.log

    print("Building vivado")
    !make vivado_raw > ./build/xil/xilinx.log

    mv_results=f"mkdir outputs/{name}; cp build/xil/*.log outputs/{name}/; cp build/xil/post_route* outputs/{name}/"
    !$mv_results

    with open(f'./outputs/{name}/coefs.txt', 'w') as f:
        f.write(str(coefs))

In [ ]:
import os

for run in range(10):
    for coef_w in [12,14,16,18,19]:
        for n in [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]:
            if (run == 0) & (coef_w < 16):
                continue
                
            coefs = gen_coefs(coef_w,n)

            for algo in ['RSG','RAGn', 'Hcub', 'HcubShallow']:
                run_test(algo, n, coefs, coef_w, run)

In [ ]:
slice_regs = []

for run in range(10):
    for coef_w in range(12,27,2):
        for n in [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]:
            for algo in ['RSG','RAGn', 'Hcub', 'HcubShallow']:

                name = f'mcm_{algo}_n{n}_width{coef_w}_run{run}'
                
                lut_cmd = f'grep "CLB LUTs" outputs/{name}/post_route_util.rpt | cut -d"|" -f3'
                lut_result=!$lut_cmd
                reg_cmd = f'grep "CLB Registers" outputs/{name}/post_route_util.rpt | head -n 1 | cut -d"|" -f3'
                reg_result=!$reg_cmd
                clb_cmd = f'grep "| CLB  " outputs/{name}/post_route_util.rpt | head -n 1 | cut -d"|" -f3'
                clb_result=!$clb_cmd

                try:
                    slice_regs.append({
                        'run': run,
                        'coef_w': coef_w,
                        'n': n,
                        'algo': algo,
                        'luts' : int(lut_result.s),
                        'regs' : int(reg_result.s),
                        'clbs' : int(clb_result.s)
                    })
                except:
                    print(f"Couldn't parse result for {name}. Did it fail synthesis?")

In [ ]:
import pandas as pd
df = pd.DataFrame(slice_regs)

In [ ]:
df

In [ ]:
import plotly.express as px
px.line(df[df['coef_w']==15], x='n',y='clbs', color='algo')